This is demonstration only.

In [39]:
%reload_ext autoreload
%autoreload 2

import json as js  # name conflict with sqla
import sqlalchemy as sqla
from sqlalchemy.orm import sessionmaker

from mpp.models import Response
# from semproc.unique_identifiers import IdentifierExtractor

import re
from semproc.bag_parser import BagParser
from semproc.nlp_utils import load_token_list
from semproc.utils import unquote, break_url, match
from semproc.xml_utils import extract_items
import dateutil.parser as dateparser
from datetime import datetime
from itertools import chain, izip
from rfc3987 import parse as uparse

import warnings
warnings.filterwarnings('ignore')

In [40]:
# for sorting out excludes

_pattern_set = [
    ('url', re.compile(ur"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))""", re.IGNORECASE)),
    # a urn that isn't a url
#     ('urn', re.compile(ur"(?!(?:http://))(?!(?:https://))(?!(?:ftp://))(([a-z0-9.\S][a-z0-9-.\S]{0,}\S:{1,2}\S)+[a-z0-9()+,\-.=@;$_!*'%/?#]+)(?=.*/)", re.IGNORECASE)),
#     ('urn', re.compile(ur"(([a-z0-9.\S][a-z0-9-.\S]{0,}\S:{1,2}\S)+[a-z0-9()+,\-.=@;$_!*'%/?#]+)(?=.*/)", re.IGNORECASE)),
    # ('urn', re.compile(ur"\burn:[a-z0-9][a-z0-9-]{0,31}:[a-z0-9()+,\-.:=@;$_!*'%/?#]+", re.IGNORECASE)),
    # this one is okay
    #('urn', re.compile(ur'([a-z0-9]{0,}:[a-z0-9][a-z0-9-]{0,31}:[a-z0-9()+,\-.:=@;$_!*\'%/?#]+)', re.IGNORECASE)),
    ('urn', re.compile(ur'^([a-z0-9.#]{0,}:([a-z0-9][a-z0-9.-]{0,31}):[a-z0-9A-Z_()+,-.:=@;$!*\'%/?#\[\]]+)', re.IGNORECASE)),
    ('uuid', re.compile(ur'([a-f\d]{8}(-[a-f\d]{4}){3}-[a-f\d]{12}?)', re.IGNORECASE)),
    ('doi', re.compile(ur"(10[.][0-9]{4,}(?:[/][0-9]+)*/(?:(?![\"&\\'])\S)+)", re.IGNORECASE)),
    ('md5', re.compile(ur"(?=(\b[A-Fa-f0-9]{32}\b))", re.IGNORECASE))
]

_rule_set = [
    ('uri', 'fileIdentifier/CharacterString'),  # ISO
    ('uri', 'identifier/'),
    ('uri', 'dataSetURI/CharacterString'),
    ('uri', 'parentIdentifier/CharacterString'),
    ('uri', 'Entry_ID'),  # DIF
    ('uri', 'dc/identifier'),  # DC
    ('basic', 'Layer/Name'),  # WMS
    ('basic', 'dataset/ID'),  # THREDDS
    ('uri', '@URI'),  # ddi
    ('uri', '@IDNo'),  # ddi
    ('uri', '@ID')
]


class Identifier(object):
    def __init__(
            self,
            tag,
            extraction_type,
            match_type,
            original_text,
            potential_identifier):
        self.tag = tag
        self.extraction_type = extraction_type
        self.match_type = match_type
        self.original_text = original_text
        self.potential_identifier = potential_identifier

    def __eq__(self, other):
        return (isinstance(other, self.__class__)
                and self.__dict__ == other.__dict__)

    def __ne__(self, other):
        return not self.__eq__(other)

    def has(self, comparison_identifier):
        return self.potential_identifier == comparison_identifier

    def __repr__(self):
        return '<Identifier %s>' % js.dumps(self.__dict__)

    def to_json(self):
        return {
            "tag": self.tag,
            "extraction_type": self.extraction_type,
            "match_type": self.match_type,
            "original_text": self.original_text,
            "potential_identifier": self.potential_identifier
        }


class IdentifierExtractor(object):
    def __init__(
            self,
            source_url,
            source_xml_as_str,
            equality_excludes=[],
            contains_excludes=[],
            tag_excludes=[]
        ):
        self.source_url = source_url
        self.source_xml_as_str = source_xml_as_str
        self.identifieds = []
        self.texts = [('', self.source_url)]
        self.seen_texts = []
        self.equality_excludes = equality_excludes
        self.contains_excludes = contains_excludes
        self.tag_excludes = tag_excludes

        self._parse()

    def _parse(self):
        try:
            parser = BagParser(self.source_xml_as_str, True, False)
        except Exception as ex:
            print ex
            raise ex
        if not parser or parser.parser.xml is None:
            raise Exception('failed to parse')

        for tag, txt in parser.strip_text():
            # do not split if it comes form an identifier field
            self.texts += ((tag, t) for t in txt.split()) if not any(r[1] in tag for r in _rule_set) else [(tag, txt)]           
            
    def _strip_punctuation(self, text):
        terminal_punctuation = '(){}[].,~|":&-<>'
        text = text.strip(terminal_punctuation)
        return text.strip()

    def _strip_dates(self, text):
        # this should still make it an invalid date
        # text = text[3:] if text.startswith('NaN') else text
        def try_format(fmt):
            try:
                d = datetime.strptime(text, fmt)
            except:
                return False
            return True
                
        try:
            d = dateparser.parse(text)
            return ''
        except ValueError:
            pass
        
        known_formats = ['%d/%m/%Y [%H:%M:%S:%f]']
        tests = [try_format(kf) for kf in known_formats]
        return '' if sum(tests) > 0 else text

    def _strip_scales(self, text):
        scale_pttn = ur"(1:[\d]{0,}(,[\d]{3}){1,})"
        m = match(text, scale_pttn)
        if m:
            return ''
        return text

    def _strip_excludes(self, match_type, tag, text):
        # TODO: how to handle the excludes for skipping
        #       by tag and skipping by value and what was
        #       the thinking a few months ago?
        
        # if it came from any known elem/attrib
        if any([t.lower() in tag.lower() for t in self.tag_excludes]):
            return ''
        
        # if it's a url, must equal an item
        if match_type == 'url':
            if text in self.equality_excludes:
                # equality only
                return ''
            return text

        # if it's *not* a url and contains any of these patterns
        if any(e.lower() in text.lower() for e in self.contains_excludes):
            return ''
        return text

    def _strip_whitespace(self, text):
        space_pattern = re.compile(' ')
        if space_pattern.subn(' ', text)[1] > 0:
            # i actually don't know if this is the right index. huh.
            return text.split(' ')[0]
        return text

    def _tidy_text(self, text):
        text = self._strip_punctuation(text)
        if not text:
            return ''

        text = self._strip_scales(text)
        if not text:
            return ''

        text = self._strip_dates(text)
        if not text:
            return ''

        text = self._strip_whitespace(text)
        if not text:
            return ''

        return text

    def _verify_urn(self, potential_urn):
        parts = re.split(r'[.:]', potential_urn)
        match_pttn = ur'\w*\b-?\d\s*\w*'
        for p in parts:
            captures = re.findall(match_pttn, u' {0} '.format(text))

            # strip them out
            if captures:
                text = re.sub('|'.join(captures), ' ', text)
                return '' if text == '0' else text

        return potential_urn if text.strip() else ''

    def _verify_url(self, potential_url):
        # ugh
        if 'mailto:' in potential_url:
            return ''
        
        # is it a urn?
        pttn = re.compile(ur'^([a-z0-9]{0,}:[a-z0-9][a-z0-9-]{0,31}:[a-z0-9()+,\-.:=@;$_!*\'%/?#\[\]]+)', re.IGNORECASE)
        urn_m = match(potential_url, pttn)
        if urn_m:
#             print 'it\'s an urn! ', potential_url, urn_m
            return ''

        # does it even have a scheme?
        try:
            u = uparse(potential_url, rule='URI')
            if not u.get('scheme'):
                return ''
        except Exception as ex:
            return ''

        pttn = re.compile(ur"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))""",
            re.IGNORECASE)
        m = match(potential_url, pttn)

        return potential_url if m else ''

    def _extract_url(self, text):
        # but really first, is it a urn?
        text = self._verify_url(text)
        if not text:
            return '', []
        url = self._tidy_text(unquote(text))
        base_url, values = break_url(url)
        values = values.split(' ') + [base_url] if base_url else []

        # return the original extracted url, and the values plus 
        # the base_url for more extracting
        return url, filter(None, [self._tidy_text(v) for v in values])

    def _extract_identifiers(self, text):
        # make sure it's not a date first
        text = self._strip_dates(text)
        if not text:
            yield '', ''
        for pattern_type, pattern in _pattern_set:
            m = match(text, pattern)
            if not m:
                continue
            if pattern_type != 'url':
                try:
                    # if the original is a URL
                    # it's a false positive URN.
                    u = uparse(text, rule='URI')
                    continue
                except:
                    pass
            if pattern_type=='url':
                # see if it's an urn
                m = self._verify_url(m)
                if not m:
                    continue
            if pattern_type=='urn':
                # check for all digits and : or . (catching bad times)
                m = self._verify_urn(m)
                print 'urn: "{0}"'.format(m)
                if not m:
                    continue
        
            yield pattern_type, m

    def _check(self, comparison_identifier, arr):
        return len([a for a in arr if a.has(comparison_identifier)]) > 0

    def process_text(self):
        while self.texts:
            tag, text = self.texts.pop()
            if self._check(text, self.seen_texts) or not text.strip():
                continue

            try:
                j = json.loads(text)
                j.keys()  # it will decode a quoted string without error
                continue
            except:
                pass
            
            # check the tag against the rule set 
            # we don't need to worry about xpaths
            # just a bit of string matching - we've
            # extracted the tags
            if any(r[1] in tag for r in _rule_set):
#                 print 'found rule set value'
                if not self._check(text, self.identifieds) \
                        and not self._check(text, self.seen_texts):
                    identify = Identifier(tag, 'rule_set', 'text', text, text)
                    self.identifieds.append(identify)
                    self.seen_texts.append(identify)
                    continue
                    

            url, values = self._extract_url(text)
            values = [v for v in values if not self._check(v, self.seen_texts)]
            self.texts += list(iter(izip([tag] * len(values), values)))
                
            if url and not self._check(url, self.identifieds) \
                    and not self._check(url, self.seen_texts):
                identify = Identifier(tag, 'extract', 'url', text, url)
                self.identifieds.append(identify)
                self.seen_texts.append(identify)

            # now run the OTHER regex
            for match_type, match_text in self._extract_identifiers(text):
                if not match_type and not match_text:
                    continue
                    
                # clean it up (make sure it's still a good thing)
                # before checking against knowns and appending
                cleaned_text = self._tidy_text(match_text)
                if cleaned_text and not self._check(
                        cleaned_text, self.identifieds):
                    self.identifieds.append(
                        Identifier(
                            tag, 'regex', match_type, text, cleaned_text
                        )
                    )
                if not self._check(cleaned_text, self.seen_texts):
                    self.texts.append((tag, cleaned_text))
                    self.seen_texts.append(
                        Identifier(
                            tag, 'regex', match_type, text, cleaned_text)
                    )
                    
        # exclude from here
        for i in self.identifieds:
            # type tag text
            text = self._strip_excludes(i.match_type, i.tag, i.potential_identifier)
            if text:
                yield i
        

In [41]:
# exclude lists

# the equality excludes 
equality = []
files = ['namespaces.txt']
for f in files:
    equality += load_token_list(f)
    
# equality = ['http://standards.iso.org/ittf/PubliclyAvailableStandards/ISO_19139_Schemas/resources/Codelist/gmxCodelists.xml']
    
contains = []
files = ['cat_interop_urns.txt', 'mimetypes.txt', 'namespaces.txt', 'excludes_by_contains.txt']
for f in files:
    contains += load_token_list(f)

tag_paths = ['@codeList', '@schemaLocation']

In [4]:
def strip_excludes(match_type, tag, text):
    # TODO: how to handle the excludes for skipping
    #       by tag and skipping by value and what was
    #       the thinking a few months ago?

    # if it came from any known elem/attrib
    if any([t.lower() in tag.lower() for t in tag_paths]):
        return ''

    # if it's a url, must equal an item
    if match_type == 'url':
        if text in equality:
            # equality only
            return ''
        return text

    # if it's *not* a url and contains any of these patterns
    if any(e.lower() in text.lower() for e in contains):
        return ''
    return text

strip_excludes(
    'url',
    'MD_Metadata/dataQualityInfo/DQ_DataQuality/scope/DQ_Scope/level/MD_ScopeCode',
    'http://standards.iso.org/ittf/PubliclyAvailableStandards/ISO_19139_Schemas/resources/Codelist/gmxCodelists.xml'
)

''

In [42]:
# grab the clean text from the rds
with open('../local/big_rds.conf', 'r') as f:
    conf = js.loads(f.read())

# our connection
engine = sqla.create_engine(conf.get('connection'))
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [5]:
response_id = 599943
response_url, response_cleaned, = session.query(
    Response.source_url,
    Response.cleaned_content
).filter(Response.id==response_id).first()

In [21]:
extractor = IdentifierExtractor(response_url, response_cleaned, equality, contains, tag_paths)

In [18]:
extractor._parse()

In [13]:
extractor.texts

[('',
  u'http://tds.ucar.edu/thredds/esgcet/65/ucar.cgd.ccsm4.CESM_CAM5_BGC_LE.ocn.proc.monthly_ave.ATM_ALT_CO2.v4.xml'),
 ('catalog/@name', u'TDS'),
 ('catalog/@name', u'configuration'),
 ('catalog/@name', u'file'),
 ('catalog/@version', u'1.0.1'),
 ('catalog/service/@name', u'TestDAPatNCAR'),
 ('catalog/service/@serviceType', u'OPENDAP'),
 ('catalog/service/@base', u'/thredds/dodsC/'),
 ('catalog/service/property/@name', u'requires_authorization'),
 ('catalog/service/property/@value', u'false'),
 ('catalog/service/property/@name', u'application'),
 ('catalog/service/property/@value', u'Web'),
 ('catalog/service/property/@value', u'Browser'),
 ('catalog/service/@name', u'fileService'),
 ('catalog/service/@serviceType', u'Compound'),
 ('catalog/service/service/@name', u'GRIDFTPatNCAR'),
 ('catalog/service/service/@serviceType', u'GridFTP'),
 ('catalog/service/service/@base', u'gsiftp://vetsman.ucar.edu:2811/'),
 ('catalog/service/service/property/@name', u'requires_authorization'),
 (

In [22]:
for i in extractor.process_text():
    print i.to_json()

{'original_text': u'ucar.cgd.ccsm4.CESM_CAM5_BGC_LE.ocn.proc.monthly_ave.ATM_ALT_CO2.v4.b.e11.B20TRC5CNBDRD.f09_g16.030.pop.h.ATM_ALT_CO2.192001-200512.nc', 'extraction_type': 'rule_set', 'tag': 'catalog/dataset/dataset/@ID', 'match_type': 'text', 'potential_identifier': u'ucar.cgd.ccsm4.CESM_CAM5_BGC_LE.ocn.proc.monthly_ave.ATM_ALT_CO2.v4.b.e11.B20TRC5CNBDRD.f09_g16.030.pop.h.ATM_ALT_CO2.192001-200512.nc'}
{'original_text': u'ucar.cgd.ccsm4.CESM_CAM5_BGC_LE.ocn.proc.monthly_ave.ATM_ALT_CO2.v4.b.e11.BRCP85C5CNBDRD.f09_g16.024.pop.h.ATM_ALT_CO2.200601-208012.nc', 'extraction_type': 'rule_set', 'tag': 'catalog/dataset/dataset/@ID', 'match_type': 'text', 'potential_identifier': u'ucar.cgd.ccsm4.CESM_CAM5_BGC_LE.ocn.proc.monthly_ave.ATM_ALT_CO2.v4.b.e11.BRCP85C5CNBDRD.f09_g16.024.pop.h.ATM_ALT_CO2.200601-208012.nc'}
{'original_text': u'ucar.cgd.ccsm4.CESM_CAM5_BGC_LE.ocn.proc.monthly_ave.ATM_ALT_CO2.v4.b.e11.BRCP85C5CNBDRD.f09_g16.025.pop.h.ATM_ALT_CO2.200601-208012.nc', 'extraction_type':

In [43]:
# let's run some identifiers (a few k to see what borks, etc)

# session.rollback()

with open('../local/local_pg.conf', 'r') as f:
    local_conf = js.loads(f.read())

# our connection
local_engine = sqla.create_engine(local_conf.get('connection'))
local_Session = sessionmaker()
local_Session.configure(bind=local_engine)
local_session = local_Session()

sketchy_sql ='''WITH t AS (SELECT *, row_number() OVER () AS rn FROM responses WHERE format = 'xml')
SELECT * FROM (
    SELECT trunc(random() * (SELECT max(rn) FROM t))::int + 1 AS rn
    FROM   generate_series(1, 200) g
    ) r
JOIN   t USING (rn);'''

In [44]:
# the model
import sqlalchemy as sqla
from sqlalchemy import (
    MetaData,
    Column,
    String,
    DateTime,
    Integer,
    Boolean,
    ForeignKey,
)
from sqlalchemy.dialects.postgresql import *
from sqlalchemy.orm import relationship, backref
from sqlalchemy.ext.declarative import declarative_base

local_Base = declarative_base()

class Extracted(local_Base):
    __tablename__ = 'identifiers'
    id = Column(Integer, primary_key=True)
    source_url = Column(String)
    tag = Column(String)
    extraction_type = Column(String)
    match_type = Column(String)
    original_text = Column(String)
    potential_identifier = Column(String)
    response_id = Column(Integer)

In [45]:
fails = []
for result in session.execute(sketchy_sql):
#     print result['id']
    identifiers = []
    try:
        extractor = IdentifierExtractor(result['source_url'], result['cleaned_content'], equality, contains, tag_paths)
        for i in extractor.process_text():
            identifiers.append(i.to_json())
    except:
        fails.append(result['id'])
        continue
    
    for e in extractor.process_text():
        ex = Extracted(
            source_url=result['source_url'],
            tag=e.tag,
            extraction_type=e.extraction_type,
            match_type=e.match_type,
            original_text=e.original_text,
            potential_identifier=e.potential_identifier,
            response_id=result['id']
        )
        try:
            local_session.add(ex)
            local_session.commit()
        except Exception as a:
            print 'failed identifier:', e.to_json()
            print a
            local_session.rollback()
            continue
        
        
len(fails)        

22

In [34]:
fails




[243851, 380749, 431926, 190880, 201042, 693828]

In [ ]:
for e in extractor.identifieds:
    print e.to_json()

In [ ]:
len(extractor.identifieds)

regex issues 

first leading 's' issue:
"SC:MOD11_L2.004:2009533550" : "C:MOD11_L2.004:2009533550" (urn)

dropping ftp on the url:
"ftp://coast.noaa.gov/pub/DigitalCoast/lidar1_z/geoid12a/data/1419" : "url" : "coast.noaa.gov/pub/DigitalCoast/lidar1_z/geoid12a/data/1419"


drop email first:

"apfo.sales@slc.usda.gov" : "url" : "usda.gov"

second leading 's' issue:
"spase://VMO/NumericalData/SAMBA/Escudero/Magnetometer/PT1S" : "urn" : "ase://VMO/NumericalData/SAMBA/Escudero/Magnetometer/PT1S"


md5 should be in whitespace or punctuation:
"f3f1ab2412999a87692116c2102bf9795c31e48dd231afced41bf444b7b5c0ed" : "md5" : "f3f1ab2412999a87692116c2102bf979"

many problems here:
"type:opendap.grid:http://stellwagen.er.usgs.gov:80/opendap/DEEP_REEF/6413pt-a.nc" : "urn" :  "ype:opendap.grid:http://stellwagen.er.usgs.gov:80/opendap/DEEP_REEF/6413pt-a.nc"

not it's not:
"/DEEP_REEF/6413pt-a.nc" : "url" : "6413pt-a.nc"

okay:
"http://hdl.handle.net/10022/AC:P:13943" : "urn" : "dl.handle.net/10022/AC:P:13943"


not a urn??
"info:oai:www.esrl.noaa.gov:728";"www.esrl.noaa.gov"


"regex";"url";"dans:oai:easy.dans.knaw.nl:easy-dataset:54740";"easy.dans.knaw.nl";585468

urn from the longer identifier non-urn thing

"UR:10:DsShESDTUR:UR:15:DsShSciServerUR:13";"UR:10:DsShESDTUR:UR:15:DsShSciServerUR:13:[EDC:DSSDSRV]:24:BR:Browse.001:2064368589"



"^([a-z0-9_#]{0,}:[a-z0-9_][a-z0-9-_]{0,31}:[a-z0-9()+,\-.:=@;$_!*'%/?#]+)"
for this: #1_SMP_Fri_Jul_20_01:55:29_UTC_2012


In [15]:
uparse("http://standards.iso.org/ittf/PubliclyAvailableStandards/ISO_19139_Schemas/resources/Codelist/gmxCodelists.xml#MD_ScopeCode", rule='URI')

{'authority': 'standards.iso.org',
 'fragment': 'MD_ScopeCode',
 'path': '/ittf/PubliclyAvailableStandards/ISO_19139_Schemas/resources/Codelist/gmxCodelists.xml',
 'query': None,
 'scheme': 'http'}

In [50]:
u = uparse("srm://esg.lbl.gov:6288/srm/v2/server?SFN=/garchive.nersc.gov/", rule='URI')
u.get('scheme')

'srm'

In [27]:
def _verify_url(potential_url):
    # ugh
    # is it a urn?
    pttn = re.compile(ur'([a-z0-9]{0,}:[a-z0-9][a-z0-9-]{0,31}:[a-z0-9()+,\-.:=@;$_!*\'%/?#]+)', re.IGNORECASE)
    urn_m = match(potential_url, pttn)
    if urn_m:
        print 'it\'s an urn! ', potential_url, urn_m
        return ''

    # does it even have a scheme?
    try:
        u = uparse(potential_url, rule='URI')
        if not u.get('scheme'):
            return ''
    except Exception as ex:
        return ''
    
    pttn = re.compile(ur"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))""",
        re.IGNORECASE)
    m = match(potential_url, pttn)

    return potential_url if m else ''

_verify_url("UR:10:DsShESDTUR:UR:15:DsShSciServerUR:13:[EDC:DSSDSRV]:24:BR:Browse.001:2064368589")

it's an urn!  UR:10:DsShESDTUR:UR:15:DsShSciServerUR:13:[EDC:DSSDSRV]:24:BR:Browse.001:2064368589 UR:10:DsShESDTUR:UR:15:DsShSciServerUR:13:


''

In [ ]:
"1984-12-31T23:00:00.000"

In [31]:
any(r[1] in 'dc/identifier' for r in _rule_set)

True

In [20]:
re.split(r'[.:]', '00:00.000')

['00', '00', '000']

In [27]:
def _verify_urn(potential_urn):
    parts = re.split(r'[.:]', potential_urn)
    match_pttn = ur'\w*\b-?\d\s*\w*'
    text = ''
    for p in parts:
        captures = re.findall(match_pttn, u' {0} '.format(p))

        # strip them out
        if captures:
            p = re.sub('|'.join(captures), ' ', p)
            text += '' if p == '0' else p

    return potential_urn if text.strip() else ''


_verify_urn('01.456.:00.000')

[u'01']
 
 
[u'456']
 
  
[]
[u'00']
 
   
[u'000']
 
    


''